<a href="https://colab.research.google.com/github/stanleyhuang12/ds542-deep-learning/blob/main/discussion_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DS 542 Spring 2026 Discussion 3


In this notebook, we will explore the use of BU's Shared Computing Cluster (SCC) by training a simple neural network on the Pima Indians Diabetes dataset.

You can read more about the dataset on [our Github repo](https://github.com/DL4DS/sp2026_discussions/blob/main/datasets/pima-indians-diabetes-dataset/README.md) and on the SCC at `/projectnb/dl4ds/materials/datasets/pima-indians-diabetes-dataset/README.md`

Please [discussion_3_setu.md](https://github.com/DL4DS/sp2026_discussions/blob/main/discussion_3_setup.md) for set up instructions.

## Instructions

1. Run your copy notebook in Jupyter on the SCC using at least one GPU as explained in the SCC lecture.
2. Confirm that you have GPU access using the code below.
3. Complete the 7 exercises included in this notebook.
4. Submit just this notebook to Gradescope.

In [ ]:
import os
import math
import socket
import psutil
import getpass
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio.v2 as imageio
from sklearn.model_selection import train_test_split

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
torch.__version__

'2.9.0+cpu'

## Part One: SCC Basics

### Exercise 1: Create a SCC session

In [ ]:
# TODO: run this block to verify your scc environment

def verify_scc_environment():

    verification_info = {
        'hostname': socket.gethostname(),
        'username': getpass.getuser(),
        'current_time': datetime.datetime.now().isoformat(),
        'python_path': os.path.dirname(os.__file__),
        'working_directory': os.getcwd(),
        'cuda_available': torch.cuda.is_available(),
        'cuda_device_count': torch.cuda.device_count() if torch.cuda.is_available() else 0,
        'cpu_count': psutil.cpu_count(),
        'memory_total_gb': round(psutil.virtual_memory().total / (1024**3), 2),
        'pytorch_version': torch.__version__
    }

    if 'scc' not in verification_info['hostname'].lower() and 'cluster' not in verification_info['hostname'].lower():
        print("Fail.")

    for key, value in verification_info.items():
        print(f"{key}: {value}")

    return verification_info

verify_info = verify_scc_environment()

Fail.
hostname: c50994dccb4e
username: root
current_time: 2026-02-06T16:17:48.819920
python_path: /usr/lib/python3.12
working_directory: /content
cuda_available: False
cuda_device_count: 0
cpu_count: 2
memory_total_gb: 12.67
pytorch_version: 2.9.0+cpu


### Exercise 2: Create a Tensor on GPU

In [ ]:
# TODO: create a tensor on GPU using `device='cuda:0`

t_gpu = torch.tensor("cuda:0")
print(t_gpu)

### Exercise 3: Impleting a CPU hard task on GPU device

In [ ]:
n = 10000
i, k = 1234, 5678


# Build A, B on GPU via broadcasting
device = torch.device(device="cuda:0")
rows = torch.arange(1, n+1, device=device, dtype=torch.float64).view(-1, 1)
cols = torch.arange(1, n+1, device=device, dtype=torch.float64).view(1, -1)

A = rows + cols                    # A_{ij} = i + j
B = cols.transpose(0,1) - cols     # B_{jk} = j - k


# TODO: multiply on GPU
C = rows @ cols  # 10k x 10k matrix multiplication, which is fast on GPU but painful on CPU.

val_gpu = C[i-1, k-1].item()
print(int(val_gpu))

### Exercise 4: Read dataset from shared folder

Read the dataset from: `/projectnb/dl4ds/materials/datasets/pima-indians-diabetes-dataset/diabetes.csv`

In [ ]:
# TODO: run this block to import the CSV file from the shared folder

df = pd.read_csv("/projectnb/dl4ds/materials/datasets/pima-indians-diabetes-dataset/diabetes.csv")
df.head()

### Exercise 5: Train a Neural Network with above dataset

In [ ]:
X = torch.tensor(df.drop("Outcome", axis=1).values, dtype=torch.float32)
y = torch.tensor(df["Outcome"].values, dtype=torch.float32).view(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# define a 2-layer NN
class ShallowNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(X.shape[1], 8)
        self.fc2 = nn.Linear(8, 2)
        self.fc3 = nn.Linear(2, 1)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))


#TODO: optimize/run the model by changing any params
model = ShallowNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

losses_nn = []


# Training
for epoch in range(5):
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    losses_nn.append(loss.item())

# Predictions on the full dataset
with torch.no_grad():
    final_predictions_nn = model(X)

# Accuracy
predicted_classes_nn = (final_predictions_nn > 0.5).float()
accuracy_nn = (predicted_classes_nn == y).float().mean()
print(f"Neural Network Accuracy: {accuracy_nn:.4f}")


# Plot
plt.figure(figsize=(12, 5))

plt.plot(losses_nn)
plt.title('Neural Network Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.tight_layout()
plt.show()

## Part Two: Train and Analyze a Neural Network

### Background

This notebook builds a model detecting trees in images.
The data set consists of roughly 500 pictures and is currently stored in `/projectnb/dl4ds/materials/datasets/tree-or-not` on the SCC.
Most of them are from the Boston area, but some are from around the globe.
Most of them were taken outside, but some were taken inside or in more exotic locations.
Many other factors such as lighting, weather, and confounding bushes will make this a challenging problem.

In [ ]:
def to_gpu(t):
    if torch.cuda.is_available():
        return t.cuda()
    return t

def to_numpy(t):
    return t.detach().cpu().numpy()

device = to_gpu(torch.ones(1,1)).device
device

### Exercise 6: Load Data

This code originally relied on checking out or cloning [this repository](https://github.com/dl4ds/fa2024_midterm).

In [ ]:
# TODO;
# - Modify this code to load from `/projectnb/dl4ds/materials/datasets/tree-or-not` (this will be the base_dir).
# - Set your image width to be 64 for faster training
# - Call the load_data_set function on both "train" and "validation"

image_width = 64
base_dir = "/projectnb/dl4ds/materials/datasets/tree-or-not"

def load_data_set(data_set_name):
    labels_path = os.path.join(base_dir, f"{data_set_name}.tsv")
    labels = pd.read_csv(labels_path, sep="\t")

    file_names = []
    images = []
    targets = []
    for i in range(labels.shape[0]):
        row = labels.iloc[i]
        image_path = os.path.join(base_dir, f"images{image_width}", row["filename"])
        try:
            image = imageio.imread(image_path)
        except:
            print("SKIPPING ", row['filename'], "MISSING")
            continue

        if image.shape[0] != image.shape[1] * 3 // 4:
            print("SKIPPING ", row['filename'], image.shape)
            continue

        # convert from 0-255 to 0.0-1.0
        image = image / 255
        # prepend axis with length one
        # image = image.reshape(1, *image.shape)
        image = torch.tensor(image, device=device, dtype=torch.float32)
        # permute image dimensions to put color channel first
        image = torch.permute(image, [2, 0, 1])

        file_names.append(row['filename'])
        images.append(image)
        targets.append(row["target"])

    images = torch.stack(images)

    targets = torch.tensor(targets, device=device, dtype=torch.float32)
    targets = targets.long()

    return (file_names, images, targets)

train_data_set = load_data_set("train.tsv")
for t in train_data_set[1:]:
    print("TRAIN", t.shape, t.dtype, t.device)
(train_file_names, train_X, train_Y) = train_data_set

validation_data_set = load_data_set("validation.tsv")
for t in validation_data_set[1:]:
    print("VALIDATION", t.shape, t.dtype, t.device)
(validation_file_names, validation_X, validation_Y) = validation_data_set

Run the following cell to preview one of the images

In [ ]:
plt.imshow(to_numpy(torch.permute(train_X[0,:,:,:], (1, 2, 0))))

### Model Building

Run the following code cells. Do not modify any of this code.

In [ ]:
class TreeNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_0 = torch.nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5, stride=2, device=device)
        self.conv_1 = torch.nn.Conv2d(in_channels=5, out_channels=5, kernel_size=3, stride=2, device=device)
        self.conv_2 = torch.nn.Conv2d(in_channels=5, out_channels=5, kernel_size=1, device=device)
        self.fc_3 = torch.nn.Linear(700, 2)

        self.relu = torch.nn.ReLU()

    def forward(self, X):

        X = self.conv_0(X)
        X = self.relu(X)

        X = self.conv_1(X)
        X = self.relu(X)

        X = self.conv_2(X)
        X = self.relu(X)

        # flatten channels and image dimensions
        X = X.reshape(X.shape[:-3] + (-1,))

        X = self.fc_3(X)

        return X

test_model = TreeNetwork().to(device)
test_output = test_model(train_X[:5,:,:,:])
assert test_output.shape == (5, 2)
del test_output

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
DEFAULT_EPOCHS = 1000 if torch.cuda.is_available() else 100

def train_model(model_class, epochs=DEFAULT_EPOCHS, learning_rate=1e-4, **kwargs):
    model = model_class(**kwargs)
    try:
        model = model.cuda()
    except:
        print("cuda() failed")
    model = torch.nn.DataParallel(model)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for i in range(epochs):
        model.train()

        optimizer.zero_grad(set_to_none=True)
        prediction = model(train_X)
        loss = loss_function(prediction, train_Y)
        loss.backward()
        optimizer.step()

        if (i + 1) % 50 == 0:
            liveloss_updates = {}
            with torch.no_grad():
                model.eval()

                metrics = {}
                def get_metrics(metrics_prefix, metrics_X, metrics_Y):
                    metrics_prediction = model(metrics_X)

                    return {
                        f"{metrics_prefix}loss": loss_function(metrics_prediction, metrics_Y)
                    }

                metrics.update(get_metrics("train_", train_X, train_Y))
                metrics.update(get_metrics("val_", validation_X, validation_Y))
                print("ITER", i+1, "METRICS", metrics)

    return model

In [ ]:
tree_model = train_model(TreeNetwork, epochs=2000)

## Exercise 7: Question

Review the metrics printed above while training the model.
What trends do you notice in the training and validation losses?
Just state the trends that you see.
You do not need to explain them.

TODO:
YOUR ANSWER HERE